In [1]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
fetch_subset = lambda subset: fetch_20newsgroups(
    subset=subset, categories=categories,
    shuffle=True, random_state=42,
    remove=('headers', 'footers', 'quotes'))
train = fetch_subset('train')
test = fetch_subset('test')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegressionCV()
pipeline = Pipeline([('vec', vec), ('clf', clf)])
pipeline.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [3]:
import eli5
from eli5 import explain_weights, explain_prediction
from eli5.formatters import format_as_html, format_as_text, format_html_styles, fields

# print(format_as_text(explain_weights(clf, vec, target_names=train['target_names'])))

/Users/kostia/shub/memex/eli5/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())

In [5]:
eli5.show_weights(clf, vec=vec, target_names=train['target_names'], horizontal_layout=False)

Weight,Feature
+18.117,atheism
+16.558,atheists
+14.393,religion
+14.380,bobby
+14.325,matthew
+13.389,motto
+13.215,atheist
+13.010,islam
+12.800,nanci
+12.216,enviroleague


In [6]:
show_html_expl(
    explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']),
    force_weights=False, horizontal_layout=True)

Weight,Feature
-1.394,<BIAS>
-14.777,Highlighted in text (sum)
Weight,Feature
+9.631,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-5.808,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


``dense_multitarget=True`` is supported for prediction explanations too, and shows just the top prediction highlighting.

In [7]:
show_html_expl(explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']),
               force_weights=True)

We can hide weights by passing ``force_weights=False`` (they still will be shown if it's impossible to highlight text)

In [8]:
show_html_expl(explain_prediction(clf, test['data'][4], vec, target_names=train['target_names']), force_weights=False)

Weight,Feature
-1.394,<BIAS>
-6.122,Highlighted in text (sum)
Weight,Feature
+7.447,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-9.098,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Show explanations for the winning class for first 10 documents from test data

In [9]:
import numpy as np
for doc in test['data'][:10]:
    expl = explain_prediction(clf, doc, vec, target_names=train['target_names'])
    show_html_expl(expl, force_weights=False)

Weight,Feature
-1.394,<BIAS>
-3.056,Highlighted in text (sum)
Weight,Feature
-1.015,<BIAS>
-3.672,Highlighted in text (sum)
Weight,Feature
+6.073,Highlighted in text (sum)
-1.016,<BIAS>
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-6.212,Highlighted in text (sum)
Weight,Feature
+7.208,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-6.282,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-14.777,Highlighted in text (sum)
Weight,Feature
+9.631,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-5.808,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-8.079,Highlighted in text (sum)
Weight,Feature
+4.294,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-5.581,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-6.122,Highlighted in text (sum)
Weight,Feature
+7.447,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.016,<BIAS>
-9.098,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-10.243,Highlighted in text (sum)
Weight,Feature
+1.509,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
+0.375,Highlighted in text (sum)
-1.016,<BIAS>
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-2.544,Highlighted in text (sum)
Weight,Feature
+0.019,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-0.512,Highlighted in text (sum)
-1.016,<BIAS>
Weight,Feature
-0.444,Highlighted in text (sum)


Weight,Feature
-1.394,<BIAS>
-8.925,Highlighted in text (sum)
Weight,Feature
-1.015,<BIAS>
-11.754,Highlighted in text (sum)
Weight,Feature
+11.409,Highlighted in text (sum)
-1.016,<BIAS>
Weight,Feature
-1.019,<BIAS>


Weight,Feature
+10.319,Highlighted in text (sum)
-1.394,<BIAS>
Weight,Feature
-1.015,<BIAS>
-6.993,Highlighted in text (sum)
Weight,Feature
-1.016,<BIAS>
-3.725,Highlighted in text (sum)
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-3.669,Highlighted in text (sum)
Weight,Feature
-1.015,<BIAS>
-7.065,Highlighted in text (sum)
Weight,Feature
+0.436,Highlighted in text (sum)
-1.016,<BIAS>
Weight,Feature
-1.019,<BIAS>


Now use a vectorizer that skips stopwords

In [10]:
vec_stop = TfidfVectorizer(stop_words='english')
clf_stop = LogisticRegressionCV()
pipeline_stop = Pipeline([('vec', vec_stop), ('clf', clf_stop)])
pipeline_stop.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

Words such as "the", "in", "of" are not used as features and are not highlighted

In [11]:
show_html_expl(explain_prediction(clf_stop, test['data'][4], vec_stop, target_names=train['target_names']), force_weights=False)

Weight,Feature
-1.395,<BIAS>
-6.399,Highlighted in text (sum)
Weight,Feature
+7.011,Highlighted in text (sum)
-1.018,<BIAS>
Weight,Feature
-1.017,<BIAS>
-6.675,Highlighted in text (sum)
Weight,Feature
-1.070,<BIAS>
